In [2]:
from sympy import *
import sympy as sy
from IPython.display import Math
# import control
print("Running Python 3.8 with sympy version", sy.__version__)
# print("Running Python 3.8 with control version", control.__version__)

Running Python 3.8 with sympy version 1.6.1


## Explaination

_Controlability_ is the ability to change the system state from the initial state to any desired states by controlling the input of the system.

_Observability_  is the ability to know the inital state of the system from the output, we only need the homogenous response to determine observability

### we use the Kalman test to determine observability and controlability


$$ Q_c =  \left[\begin{matrix}A & A B & A^{2} B & ... A^{n-1}B\end{matrix}\right]$$
 if determinant of Qc is not zero then system is controllable.



$$ Q_0 = \left[\begin{matrix} C  \\  C A  \\ \vdots \\ C A^{n-1} \end{matrix} \right] $$

or another way of writing it is 
$$ Q_0 = \left[\begin{matrix} C^T & A^T C^T  & \dots (A^T)^{n-1} C^T \end{matrix} \right] $$


 if determinant of Q0 is not zero then the system is observable. $ x^T $ means the transpose of x.



The order of a matrix is m x n where m is the number of rows and n is the number of columns. We use the order of the A matrix when solving.


If we dont have a full rank for Qc or Q0 then the system is partially controllable/ observable


In [346]:
# defining functions we will use

def get_qc(a,b):
    # order of the matrix m x n
    n = a.cols
    # qc = (b, a*b, a^2 *b, a^3 *b, ...a^(n-1) * b)
    return Matrix([[a**x * b for x in range(n)]])

def get_q0_transpose(a,c):
    '''It says transpose but this is the main way '''
    return get_q0(a,c).T
    
def get_q0(a,c):
    '''This gives you the second way of writing it'''
    n = a.cols
    return Matrix([[ a.T**x  * c.T for x in range(n)]])
        
def is_controlabale(A, B):
    qc = get_qc(A,B)
    d = qc.det()
    if d !=0 :
        return True
    return False

def is_observable(a,c):
    q0 = get_q0(a,c)
    d = q0.det()
    if d!=0:
        return True
    return False

def display_new_ABC(a,b,c, S):
    s_n = get_inverse(S)
    print("\n new A = S^-1 A S = " )
    display(s_n *a * S)
    display("new B = S^-1 B", s_n *b )
    display("new C = c S",c*S )
    
def get_inverse(m):
    '''
    Takes in a matrix t and returns the inverse assuming it exists, the inverse will have it's poles
    seperated by a gaussian dividtion by the determinent
    Params:
        m (list or sympy.Matrix)
    Returns:
        (sympy.Matrix) Inverted matrix or None if matrix doesnt exist
        
    '''
    m = sy.Matrix(m)
    if sy.det(m)!=0:
        m = m.inv()
        m = m.applyfunc(lambda e: sy.factor(e, gaussian=True))
    else:
        print("Not invertable, det=0")
        return None
    return m

def get_poles_from_matrix(a, show_steps = False):
    '''Takes the A matrix of a system and returns the poles, format is {pole: multiplicity} can be used to determine stability'''
    ss=symbols('s')
    char_mtx = ss*eye(a.rows) - a
    char_eq = char_mtx.det()
    if show_steps:
        display(char_mtx)
        display(char_eq)
    return(roots(char_eq))


## Examples
1. Determine controlability of the following:  $$\dot{x} = \left[\begin{matrix}-1 & -1\\1 & 0\end{matrix}\right]x + \left[\begin{matrix}1\\0\end{matrix}\right] u $$ 

<br>

2. Determine the observability of the following: $$ \dot{x} = \left[\begin{matrix}-2 & -2\\1 & 0\end{matrix}\right] x + \left[\begin{matrix}1\\0\end{matrix}\right] u \\ y = \left[\begin{matrix}1 & 1\end{matrix}\right] $$


<br>

3. Determine the observability and controlability of the following:
$$ A = \left[\begin{matrix}1 & 2 & -1\\0 & 1 & 0\\1 & -4 & 3\end{matrix}\right] , B = \left[\begin{matrix}0\\0\\1\end{matrix}\right], C = \left[\begin{matrix}1\\-1\\1\end{matrix}\right]
$$


In [96]:
#Q1 is the following system controllale

a = Matrix([[-1,-1],[1,0]])
b = Matrix([[1],[0]])
display("A", a)
display("B", b)
ans = is_controlabale(a,b)
display(f'is the system controllable?: {ans}')

'A'

Matrix([
[-1, -1],
[ 1,  0]])

'B'

Matrix([
[1],
[0]])

'is the system controllable?: True'

In [110]:
# Q2
a = Matrix([[-2, -2], [1,0]])
b = Matrix([[1],[0]])
c = Matrix([[1,1]])
# get_q0(a,c)
ans = is_observable(a,c)
display(f'is the system observable?: {ans}')

'is the system observable?: True'

In [120]:
#Q3
a = Matrix([
    [1,2,-1],
    [0,1,0],
    [1,-4, 3]
])
b = Matrix([[0], [0], [1]])
c = Matrix([[1,-1,1]])
display(f'is the system controllable?: {is_controlabale(a,b)}')
display(f'is the system observable?: {is_observable(a,c)}')


'is the system controllable?: False'

'is the system observable?: False'

## Controllable and/or Observable component in a non-controllable and/or non-observable system

### Finding the Controllable component

lets start by a system that is not controllable
If the system Qc matrix has a linearly dependant column then the system is not controllable, however, there is a controllable component in it. In the Qc matrix you will have linearly independent columns which are controllable.
You can determine if a column is linearly independnt by takeing the reduced row echalon form of the Qc and every column that sticks to the reduced row echalon form then it is an indepeendnt column. You take the reduced row echalon form in sympy using the method `.rref()`

The number of controllable components is equal to the rank of the Qc matrix.
Now to get the controllable component replace the columns that are not linealy independent with an arbitrary column in the QC, and call the new modified Qc the S matrix, Then we use the Similarity Transormation  to get the new A matrix , and do this to get the rest of the matrices  $$A_{new} =  S^{-1} A S \\
B_{new} = S^{-1} B \\
C_{new} = CS$$

We then put those new A,B,C matrices in this state space model

$$ \left[\begin{matrix}\dot{x_c} \\ \overline{\dot{x_{c}}} \end{matrix}\right]   = \left[ \begin{matrix} A_c && A_{c2} \\ 0 && \overline{A_c} \end{matrix} \right]
\left[\begin{matrix} x_c \\ \overline{x_{c}} \end{matrix}\right]
 + \left[ \begin{matrix} B_c \\ 0 \end{matrix} \right] u
$$
$$
Y = \left[\begin{matrix}C_c && \overline{C_c} \end{matrix}\right]
\left[\begin{matrix} x_c \\ \overline{x_{c}} \end{matrix}\right]
+ Du
$$

Where 
$x_c$ = the controllable state vector, the number of controllable vars is equal to the number of linearly independent vecotrs in the Qc

$\overline{x_c}$ the un-controllable state vector

$A_c, B_c, C_c$ the parameters of the controllable component

$\overline{A_c}, \overline{B_c}, \overline{C_c}$ the parameters of the un-controllable component



In [169]:
#Example
a = Matrix([
    [0, 1, 0],
    [0, 0, 1],
    [-6, -11, -6]
])

b = Matrix([
    [0],
    [1],
    [-3]
])
c = Matrix([[-1, 0, 2]])
display(f'is the system controllable?: {is_controlabale(a,b)}')
display("This is Qc" , get_qc(a,b))
display("From the rref we see that the last column in Qc is not linearly independent",get_qc(a,b).rref()[0])

print(" \n We get the S matrix by replacing the non-linearly dependent with an arbitary column [0,0,1]")
S = Matrix([[0, 1, 0], [1, -3, 0], [-3, 7, 1]])
display("S = ", S)

print("\n new A = S^-1 A S = " ,S**-1 *a * S )
display("new B = S^-1 B", S**-1 *b )
display("new C = c S",c*S )

print("\n Since the first two columns of Qc are linearly independent and only the 3rd col is dependent we take the first two cols and rows from the new A,B,C and as a controllable compoenent and we take the last col and row as uncontrollable")

# x1_dot, x2_dot, x3_dot = symbols("\dot{x_1}, \dot{x_2}, \dot{x_3}")
# x1, x2, x3 = symbols("x_1, x_2, x_3")

print("\n \n The answer is neatly written below")

'is the system controllable?: False'

'This is Qc'

Matrix([
[ 0,  1,  -3],
[ 1, -3,   7],
[-3,  7, -15]])

'From the rref we see that the last column in Qc is not linearly independent'

Matrix([
[1, 0, -2],
[0, 1, -3],
[0, 0,  0]])

...

'new B = S^-1 B'

Matrix([
[1],
[0],
[0]])

'new C = c S'

Matrix([[-6, 13, 2]])


 Since the first two columns of Qc are linearly independent and only the 3rd col is dependent we take the first two cols and rows from the new A,B,C and as a controllable compoenent and we take the last col and row as uncontrollable

 
 The answer is neatly written below


The controllable component is:
$$
\left[\begin{matrix}\dot{x_1}\\ \dot{x_2}\end{matrix}\right] = \left[\begin{matrix}0 & 2\\1 & 3\end{matrix}\right]\left[\begin{matrix} x_1 \\ x_2 \end{matrix}\right] + \left[\begin{matrix}1\\0\end{matrix}\right] u(t)
$$

$$
y(t) = \left[\begin{matrix}-6 & 13\end{matrix}\right] \left[\begin{matrix} x_1 \\ x_2 \end{matrix}\right]
$$


The unconntrollable component is:
$$ \dot{x_3} = -3x_3 \\ y(t) = 2x_2 $$

In [182]:
# example 2 
a = Matrix([
    [-11, -36, 0],
    [2,8,1],
    [-4,-16, -2]
])
b = Matrix([
    [4],[-1],[2]
])
c = Matrix([[1,2,-1]])
d =1
qc = get_qc(a,b)
display("qc", qc)
display(qc.rref()[0])
S = Matrix([
    [4, 1, 0],
    [-1, 0, 1],
    [2,0 , 0]
])
display("Replace the dependant cols with arbitrary cols, let S be", S)

display_new_ABC(a,b,c,S)
print("answer is neatly written below")

'qc'

Matrix([
[ 4, -8, 16],
[-1,  2, -4],
[ 2, -4,  8]])

Matrix([
[1, -2, 4],
[0,  0, 0],
[0,  0, 0]])

'Replace the dependant cols with arbitrary cols, let S be'

Matrix([
[ 4, 1, 0],
[-1, 0, 1],
[ 2, 0, 0]])

...

'new B = S^-1 B'

Matrix([
[1],
[0],
[0]])

'new C = c S'

Matrix([[0, 1, 2]])

answer is neatly written below


The controllable component is:
$$ \dot{x_1} = -2x_1 + u(t) \\ y(t) =  u(t) $$



The unconntrollable component is:
$$
\left[\begin{matrix}\dot{x_2}\\ \dot{x_3}\end{matrix}\right] = \left[\begin{matrix}-3 & -4\\0 & 0\end{matrix}\right]\left[\begin{matrix} x_2 \\ x_3 \end{matrix}\right] + \left[\begin{matrix}0\\0\end{matrix}\right] u(t)
$$
$$
y(t) = \left[\begin{matrix}1 & 2\end{matrix}\right] \left[\begin{matrix} x_2 \\ x_3 \end{matrix}\right] + u(t)
$$


### Finding the Observable compoenent

If a system is not observable we can find its obsercable component, this is similar to finding the controllable component above, with few diffrences
To find the $S^{-1}$ matrix We take the Qoand replace the dependant rows by arbitrarly rows to make $S^{-1}$ a full rank matrix
Then we find the new A,B,C the same way we do for controllability component above.

$$ \left[\begin{matrix}\dot{x_o} \\ \overline{\dot{x_{o}}} \end{matrix}\right]   = \left[ \begin{matrix} A_o && 0 \\ A_{o21} && \overline{A_o} \end{matrix} \right]
\left[\begin{matrix} x_o \\ \overline{x_{o}} \end{matrix}\right]
 + \left[ \begin{matrix} B_o \\ \overline{B_o} \end{matrix} \right] u
$$
$$
Y = \left[\begin{matrix}C_o && 0 \end{matrix}\right]
\left[\begin{matrix} x_o \\ \overline{x_{o}} \end{matrix}\right]
+ Du
$$

Where 
$x_o$ = the observable state vector, the number of observable vars is equal to the number of linearly independent vecotrs in the Qc

$\overline{x_o}$ the un-observable state vector

$A_o, B_o, C_o$ the parameters of the observable component

$\overline{A_o}, \overline{B_o}, \overline{C_o}$ the parameters of the un-controllable component


In [247]:
# Example
a = Matrix([
    [0, 1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1],
    [0,0,21.6, 0]
])
b = Matrix([
    [0],
    [1],
    [0],
    [-2]
])
c = Matrix([[0,0,1,0]])
print(f' is the system observable? {is_observable(a,c)}')
q0 = get_q0_transpose(a,c)
display("Q0 transpose (some books use transpose as the main q0)", q0)
print(f' Since the rank is {q0.rank()} we have 2 independat rows. We use the transpose to turn the rows into columns to simplify computatinos')
print("to find independatn rows using rref() we convert the rows to columns so we can find the linearly indepenadt columns of the transpose, which are indepndant rows if we reverse the transpose ")
display("From the Q0 transpose rref we see that the first two cols are independant, meaning that the first two rows of the Q0 are observable and we keep them, and we replace the last 2 rows with arbitrary values to get the the S^-1 to be a full rank marix ",q0.T.rref()[0])


print("Now to find S^-1 we replace the last 2 rows of Q0 with the arbitrarly chosen rows [0 1 0 0] and [1 0 0 0]")
s_neg = Matrix([
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 0]
])
display("S^-1 is", s_neg) 
S = get_inverse(s_neg)
display("so after taking the inverse we get S = ", S)
display_new_ABC(a,b,c, S)

print("The first two rows and columns of the new A,B,C are the observable component and the last two are the non-observable. Answer is written below")


 is the system observable? False


'Q0 transpose (some books use transpose as the main q0)'

Matrix([
[0, 0,    1,    0],
[0, 0,    0,    1],
[0, 0, 21.6,    0],
[0, 0,    0, 21.6]])

 Since the rank is 2 we have 2 independat rows. We use the transpose to turn the rows into columns to simplify computatinos
to find independatn rows using rref() we convert the rows to columns so we can find the linearly indepenadt columns of the transpose, which are indepndant rows if we reverse the transpose 


'From the Q0 transpose rref we see that the first two cols are independant, meaning that the first two rows of the Q0 are observable and we keep them, and we replace the last 2 rows with arbitrary values to get the the S^-1 to be a full rank marix '

...

'new B = S^-1 B'

Matrix([
[ 0],
[-2],
[ 1],
[ 0]])

'new C = c S'

Matrix([[1, 0, 0, 0]])

The first two rows and columns of the new A,B,C are the observable component and the last two are the non-observable. Answer is written below


The observable compoenent
$$ \left[\begin{matrix}\dot{x_1} \\ {\dot{x_{2}}} \end{matrix}\right]   = \left[ \begin{matrix} 0 && 1 \\ 21.6 && 0 \end{matrix} \right]
\left[\begin{matrix} x_1 \\ x_2 \end{matrix}\right]
 + \left[ \begin{matrix} 0 \\ -2 \end{matrix} \right] u
$$
$$
Y = \left[\begin{matrix}1 && 0 \end{matrix}\right]
\left[\begin{matrix} x_1 \\ x_{2} \end{matrix}\right]
$$

The nonobservable component
$$ \left[\begin{matrix}\dot{x_3} \\ {\dot{x_{4}}} \end{matrix}\right]   = \left[ \begin{matrix} 0 && 0 \\ 1 && 0 \end{matrix} \right]
\left[\begin{matrix} x_3 \\ x_4 \end{matrix}\right]
 + \left[ \begin{matrix} 1 \\ 0 \end{matrix} \right] u
$$
$$
Y = \left[\begin{matrix}1 && 0 \end{matrix}\right]
\left[\begin{matrix} x_3 \\ x_{4} \end{matrix}\right]
$$

In [257]:
## Example 2
a = Matrix([
    [0, 0, -6],
    [1, 0, -11],
    [0, 1, -6]
])
b = Matrix([
    [-1],
    [0],
    [2]
])
c = Matrix([[0, 1, -3]])

qo = get_q0_transpose(a,c)
display("q0", qo)
display("finding the depandant rows", qo.T.rref()[0])
print("Only the last row is depandent so replace it with the arbitrarly chosen [0 0 1] row")
s_neg = Matrix([
    qo.row(0),
    qo.row(1),
    [0,0,1]
])
s = get_inverse(s_neg)
display("S^-1", s_neg, "Taking its inverse we get S = ", s)
display_new_ABC(a,b,c,s)
print("\n answer written below")

'q0'

Matrix([
[ 0,  1,  -3],
[ 1, -3,   7],
[-3,  7, -15]])

'finding the depandant rows'

Matrix([
[1, 0, -2],
[0, 1, -3],
[0, 0,  0]])

Only the last row is depandent so replace it with the arbitrarly chosen [0 0 1] row


...

'new B = S^-1 B'

Matrix([
[-6],
[13],
[ 2]])

'new C = c S'

Matrix([[1, 0, 0]])


 answer written below


Observable component
$$ \left[\begin{matrix}\dot{x_1} \\ {\dot{x_{2}}} \end{matrix}\right]   = \left[ \begin{matrix} 0 && 1 \\ -2 && -3 \end{matrix} \right]
\left[\begin{matrix} x_1 \\ x_2 \end{matrix}\right]
 + \left[ \begin{matrix} -6 \\ 13 \end{matrix} \right] u
$$
$$
Y = \left[\begin{matrix}1 && 0 \end{matrix}\right]
\left[\begin{matrix} x_1 \\ x_{2} \end{matrix}\right]
$$

non-observable component
$$ 
\dot{x_3} = -3x_3 + 2u \\
y = 0
$$